In [1]:
import json

In [2]:
my=open('C:/Users/sai kiran Reddy/Desktop/chat/pytorch-chatbot-master/data.json','r')
jsondata=my.read()
intents=json.loads(jsondata)
print(intents)

{'intents': [{'tag': 'Greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there', 'Hello', 'Good day'], 'responses': ['Hey:-)', 'Hello, thanks for visiting', 'Hi there, What can I do for you', 'hi there, how can I help?']}, {'tag': 'Placements', 'patterns': ['Is there any Placements in your College?', 'Do we have any Placements in your College?', 'Is There Placement In College?'], 'responses': ['yes,we have good Placements in our College']}, {'tag': 'Admission', 'patterns': ['How To Apply For The Admission For Bsc, Msc, Puc, Professional, Evening College, Certification Course ?', 'How We Will Get The Admission For Bsc, Msc, Puc, Professional, Evening College, Certification Course ?', 'How To Get Admission For Bsc, Msc, Puc, Professional, Evening College, Certification Course ?'], 'responses': ['First you need apply through our college website,and then you will get a date for entrance exam,after that you have interview,if you select in that round,you will get seat in our coll

In [3]:
import json 
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
my=open('C:/Users/sai kiran Reddy/Desktop/chat/pytorch-chatbot-master/data.json','r')
jsondata=my.read()
intents=json.loads(jsondata)
stemmer = PorterStemmer()

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [4]:
all_words = []
tags = []
xy = []

In [5]:
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

52 patterns
15 tags: ['About courses', 'Admission', 'Campus', 'Canteen', 'Contact Information', 'Ending', 'Fees', 'Greeting', 'Hostel facility', 'Internship', 'Library_location', 'Outreach program', 'Placements', 'Sports activity encouragement', 'Timings']
117 unique stemmed words: ["'s", ',', 'a', 'admiss', 'all', 'allow', 'analyt', 'ani', 'anyon', 'appli', 'are', 'avail', 'back', 'be', 'befor', 'big', 'boy', 'break', 'bsc', 'by', 'bye', 'campu', 'can', 'canteen', 'certif', 'colleg', 'complet', 'contact', 'cours', 'cut', 'cut-off', 'data', 'day', 'do', 'els', 'encourag', 'entranc', 'even', 'exam', 'fee', 'food', 'for', 'from', 'get', 'girl', 'go', 'good', 'ha', 'happen', 'have', 'hello', 'hey', 'hi', 'hostel', 'how', 'if', 'in', 'insid', 'internship', 'is', 'it', 'item', 'lab', 'later', 'librari', 'locat', 'maintain', 'mani', 'month', 'msc', 'much', 'need', 'non-veg', 'noth', 'now', 'number', 'of', 'off', 'offic', 'okay', 'our', 'outreach', 'park', 'phone', 'placement', 'possibl', 'pr

In [18]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [19]:
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [20]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [21]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "Thankyou":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                reply=random.choice(intent['responses'])
                print(f"{bot_name}",reply)
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hi
Sam Hello, thanks for visiting
You: Thankyou